In [1]:
import xarray as xr
import geopandas as gpd
import numpy as np
import pendulum
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore") #ignora runtime warning por causa de nan na hora da correlacao

from skill import recorte
from skill import clima
from skill import stats
from skill import config

# Configurando diretorios

In [2]:
dir_dados = config.dir_dados
dir_img = config.dir_img
dir_shp = config.dir_shp

# Abrindo arquivos

In [3]:
gpcc = xr.open_dataset(f'{dir_dados}/precip.comb.v2020to2019-v2020monitorafter.total.nc')
br_madeira = gpd.read_file(f'{dir_shp}/brasil_madeira.shp')

## Abrindo arquivo de previsão p/ utilizar lat e lon p/ interpolacao da climatologia e do observado

In [4]:
aux = xr.open_dataset(f'{dir_dados}/cam3_init1.nc')

lat = aux.lat.values
lon = aux.lon.values

# Trabalhando com observado

## Interpola para a grade do cam

In [5]:
obs_interp = recorte.regridder(gpcc, lat, lon)
obs_br = recorte.recortar(obs_interp, br_madeira)

# Calculando RMSE para previsão bruta do CAM

In [6]:
init = np.arange(1,13,1)
leads = np.arange(1,4,1)

dir_dados.joinpath('cam3-rmse_prev_bruta').mkdir(exist_ok=True)

for i in init:
    cam = xr.open_dataset(f'{dir_dados}/cam3_init{i}.nc')
    cam['chuva'] = (cam.PRECC + cam.PRECSH + cam.PRECL)*3600*24*30*1000

    cam_br = recorte.recortar(cam.chuva, br_madeira)
    
    data = pendulum.now().set(year=1975, month=i, day=21)
    result = list()
    for lead in leads:
        
        mes = (data + relativedelta(months=lead)).month
        
        ds_cam = cam_br.sel(time=cam_br.time.dt.month == mes, lead=lead)
        ds_obs = obs_br.sel(time=ds_cam.time.values).precip
        
        rmse = stats.rmse(ds_cam, ds_obs)
        result.append(rmse)
        
    ds_rmse_br = xr.concat(result, dim='lead')
    ds_rmse_br.to_netcdf(dir_dados.joinpath('cam3-rmse_prev_bruta',f'cam3_init{i}_rmse_bruta.nc'))

# Calculando RMSE para previsão com ajuste e ponderação pelo skill

In [8]:
dir_dados.joinpath('cam3-rmse_prev_ajustada').mkdir(exist_ok=True)

In [9]:
init = np.arange(1,13,1)
leads = np.arange(1,4,1)

dir_dados.joinpath('cam3-rmse_prev_ajustada', 'com_skill').mkdir(exist_ok=True)

for i in init:
    cam_ajustado = xr.open_dataset(dir_dados.joinpath('cam3_ajuste','com_skill', f'cam3_ajuste_{i}.nc')).__xarray_dataarray_variable__
    
    data = pendulum.now().set(year=1975, month=i, day=21)
    result = list()
    for lead in leads:
        
        mes = (data + relativedelta(months=lead)).month
        
        ds_cam_ajustado = cam_ajustado.sel(time=cam_ajustado.time.dt.month == mes, lead=lead)
        ds_obs = obs_br.sel(time=ds_cam_ajustado.time.values).precip
        
        rmse = stats.rmse(ds_cam_ajustado, ds_obs)
        result.append(rmse)
        
    ds_rmse_br = xr.concat(result, dim='lead')
    ds_rmse_br.to_netcdf(dir_dados.joinpath('cam3-rmse_prev_ajustada','com_skill',f'cam3_init{i}_rmse_ajustado.nc'))

# Calculando RMSE para previsão apenas com ajuste

In [10]:
init = np.arange(1,13,1)
leads = np.arange(1,4,1)

dir_dados.joinpath('cam3-rmse_prev_ajustada', 'sem_skill').mkdir(exist_ok=True)

for i in init:
    cam_ajustado = xr.open_dataset(dir_dados.joinpath('cam3_ajuste','sem_skill', f'cam3_ajuste_{i}.nc')).__xarray_dataarray_variable__
    
    data = pendulum.now().set(year=1975, month=i, day=21)
    result = list()
    for lead in leads:
        
        mes = (data + relativedelta(months=lead)).month
        
        ds_cam_ajustado = cam_ajustado.sel(time=cam_ajustado.time.dt.month == mes, lead=lead)
        ds_obs = obs_br.sel(time=ds_cam_ajustado.time.values).precip
        
        rmse = stats.rmse(ds_cam_ajustado, ds_obs)
        result.append(rmse)
        
    ds_rmse_br = xr.concat(result, dim='lead')
    ds_rmse_br.to_netcdf(dir_dados.joinpath('cam3-rmse_prev_ajustada','sem_skill',f'cam3_init{i}_rmse_ajustado.nc'))